# Behavioural Cloning Project

## 1. Import Data from Folders

In [1]:
import csv
import os
from sklearn.model_selection import train_test_split

# Load data from file.
filename = 'data_new'
samples = os.listdir(filename)
lines = []
zeros = 0

# Loop through all directories.
for sample in samples:
    current_dir = './' + filename + '/' + sample
    with open(current_dir + '/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            
            # Filter sections of zero steering.
            if float(line[3]) != 0.0:
                zeros = 0
            else:
                zeros += 1
            
            # Append images.
            if zeros < 5:
                line[0] = current_dir + '/IMG/' + line[0].split('/')[-1]
                line[1] = current_dir + '/IMG/' + line[1].split('/')[-1]
                line[2] = current_dir + '/IMG/' + line[2].split('/')[-1]
                lines.append(line)

# Split data into training and validation set.
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

## 2. Image Augmentation Function

In [2]:
from random import randint as rand

def augment(image):
    """Image Augmentation Function
    
    Applies a random rotation, translation, and brightness
    effect to the input image.
    
    Returns:
      Image, RGB image.
    """
    # Image Parameters.
    rows,cols,depth = image.shape

    # Apply Random Translation.
    x_shift, y_shift = rand(0,5)/100, rand(0,3)/100
    M = np.float32([[1,0,cols*x_shift],[0,1,rows*y_shift]])
    image = cv2.warpAffine(image,M,(cols,rows))

    # Apply Random Rotation.
    rotate = rand(-5,5)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),rotate,1)
    image = cv2.warpAffine(image,M,(cols,rows))

    # Apply Random Brightness.
    image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    image[:,:,2] = image[:,:,2]*random_bright
    image = cv2.cvtColor(image,cv2.COLOR_HSV2RGB)
    
    return image

## 3. Populate Images and Measurements

In [3]:
import cv2
import numpy as np
import sklearn
from matplotlib import pyplot as plt
from sklearn.utils import shuffle

%matplotlib inline

def generator(samples, batch_size=32):
    """Data Generator to Prevent Overflow
    
    Returns:
      List, images of left/right/center camera.
      List, steering angles.
    """
    steering_bias_left = 0.25
    steering_bias_right = -0.25
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurements = []
            for batch_sample in batch_samples:
                # Collect Images & Measurements.
                image_c = cv2.imread(batch_sample[0])
                image_l = cv2.imread(batch_sample[1])
                image_r = cv2.imread(batch_sample[2])
                measurement_c = float(batch_sample[3])
                measurement_l = float(measurement_c) + steering_bias_left
                measurement_r = float(measurement_c) + steering_bias_right
                
                # Add Normal & Mirrored Images.
                images.append(augment(image_c))
                images.append(augment(np.fliplr(image_c)))
                images.append(augment(image_l))
                images.append(augment(np.fliplr(image_l)))
                images.append(augment(image_r))
                images.append(augment(np.fliplr(image_r)))

                # Add Normal & Mirrored Steering.
                measurements.append(measurement_c)
                measurements.append(-measurement_c)
                measurements.append(measurement_l)
                measurements.append(-measurement_l)
                measurements.append(measurement_r)
                measurements.append(-measurement_r)

            # Set data into array.
            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

# Compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

## 4. Model Architecture.

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Cropping2D, Dense
from keras.layers import Dropout, Flatten, Lambda
from keras.layers import Convolution2D

# NVIDIA Architecture.
model = Sequential()

# Pre-Process Data.
model.add(Lambda(lambda x: x / 127.5 -1, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((52,20),(15,15))))

# Add Three 5x5 Convolution Layers.
model.add(Convolution2D(24, 5, 5, subsample=(2,2), border_mode='same', activation='relu'))
model.add(Dropout(0.25))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), border_mode='same', activation='relu'))
model.add(Dropout(0.25))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), border_mode='same', activation='relu'))
model.add(Dropout(0.25))

# Add Two 3x3 Convolution Layers.
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(0.25))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(0.25))

# Flatten Layers.
model.add(Flatten())

# Connect the layers.
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(1))

# Compile the Model.
model.compile(loss='mse', optimizer='adam')

#https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/

Using TensorFlow backend.


## 5. Train the Model.

In [ ]:
history_object = model.fit_generator(
    train_generator, 
    samples_per_epoch=len(train_samples)*6, 
    validation_data=validation_generator, 
    nb_val_samples=len(validation_samples)*6,
    nb_epoch=8)

# Save Training Model.
model.save('model_data1.h5')

# Visualize the Loss.
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

Epoch 1/8
12546/12546 [==============================] - 53s - loss: 0.0548 - val_loss: 0.0558
Epoch 2/8
12546/12546 [==============================] - 48s - loss: 0.0543 - val_loss: 0.0558
Epoch 3/8
12546/12546 [==============================] - 47s - loss: 0.0531 - val_loss: 0.0486
Epoch 4/8
12546/12546 [==============================] - 46s - loss: 0.0474 - val_loss: 0.0422
Epoch 5/8
12546/12546 [==============================] - 46s - loss: 0.0430 - val_loss: 0.0386
Epoch 6/8
12480/12546 [============================>.] - ETA: 0s - loss: 0.0403